In [ ]:
import osiris_utils as ou
import numpy as np
from pathlib import Path
from tqdm import tqdm
import h5py
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import RegularGridInterpolator
from tqdm import tqdm

import contextlib
import io

plt.rcParams['font.size'] = 14

In [ ]:

sim_Boris_0_01 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_0_01/Boris_0_01.in")

sim_Boris_0_1 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_0_1/Boris_0_1.in")

sim_Boris_1 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_1/Boris_1.in")

sim_Boris_10 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_10/Boris_10.in")


sim_GcaCorr_1 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/GcaCorr/GcaCorr_1/GcaCorr_1.in")

sim_GcaCorr_10 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/GcaCorr/GcaCorr_10/GcaCorr_10.in")

In [ ]:
def call_silently(func, *args, **kwargs):
    with contextlib.redirect_stdout(io.StringIO()):
        return func(*args, **kwargs)


def get_trajectory(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["x1"][particle, :][mask],
        track.data["x2"][particle, :][mask],
        track.data["x3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def get_p(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["p1"][particle, :][mask],
        track.data["p2"][particle, :][mask],
        track.data["p3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj


def get_fields(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["E1"][particle, :][mask],
        track.data["E2"][particle, :][mask],
        track.data["E3"][particle, :][mask],
        track.data["B1"][particle, :][mask],
        track.data["B2"][particle, :][mask],
        track.data["B3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def get_gradB(sim, t=None):
    
    B = (sim["part_b1"]**2 + sim["part_b2"]**2 + sim["part_b3"]**2)**(0.5)

    dB_x1 = ou.Derivative_Diagnostic(B, "x1")
    dB_x2 = ou.Derivative_Diagnostic(B, "x2")
    dB_x3 = ou.Derivative_Diagnostic(B, "x3")

    if isinstance(t, int):
        return np.stack([
            dB_x1[t],
            dB_x2[t],
            dB_x3[t]
        ], axis=0)
    else:
        return np.stack([
            dB_x1,
            dB_x2,
            dB_x3
        ], axis=0)

def get_Jacobian_b(sim, t=None):
    
    B = (sim["part_b1"]**2 + sim["part_b2"]**2 + sim["part_b3"]**2)**(0.5)

    b1 = sim["part_b1"] / B
    b2 = sim["part_b2"] / B
    b3 = sim["part_b3"] / B

    J_B = np.empty((3, 3), dtype=object)
    for i, b_buffer in enumerate([b1, b2, b3]):
        for j, axis in enumerate(["x1", "x2", "x3"]):
            J_B[i,j] = ou.Derivative_Diagnostic(b_buffer, axis)

    if isinstance(t, int):
        return J_B[:, :, t]
    else:
        return J_B


def get_Jacobian_b_Interp(J_B, t=None):
    vfunc = np.vectorize(diag_interpolator, otypes=[object])
    return vfunc(J_B, t=t)


def diag_interpolator(diag, t=0):
    # TODO maybe considder periodic boundaries
    
    # Define grid points
    x = np.linspace(diag.grid[0][0] + diag.dx[0], diag.grid[0][1], diag.nx[0])
    y = np.linspace(diag.grid[1][0] + diag.dx[1], diag.grid[1][1], diag.nx[1])
    z = np.linspace(diag.grid[2][0] + diag.dx[2], diag.grid[2][1], diag.nx[2])
    
    return RegularGridInterpolator((x, y, z), diag[t], bounds_error=False, fill_value=None, method='cubic')



def plot_drifts(sim, particles, tmin = 0, tmax = 100000000000, unload=False, fig=None, ax=None):
    # plots non relativistic gca drifts along time

    gradB=get_gradB(sim)
    interp_gradB_1 = diag_interpolator(gradB[0], t=0)
    interp_gradB_2 = diag_interpolator(gradB[1], t=0)
    interp_gradB_3 = diag_interpolator(gradB[2], t=0)

    J_b = get_Jacobian_b(sim)
    J_b_Interp = get_Jacobian_b_Interp(J_b, t=0)
    
    v_EXB=[]
    v_GradB=[]
    v_c=[]
    v_v_EGrad_B=[]
    for particle in tqdm(particles, desc="Calculating Drifts"):
        # get x, p, E, B, t for the trajecotry of each particle
        fields = call_silently(get_fields, sim, particle, tmin, tmax)
        traj = call_silently(get_trajectory, sim, particle, tmin, tmax)
        p = call_silently(get_p, sim, particle, tmin, tmax)

        E = fields[0:3, :]
        B = fields[3:6, :]
        b = B / np.linalg.norm(B, axis=0)
        b_B = b / np.linalg.norm(B, axis=0)
        t = fields[-1, :]


        # E x B drift
        v_EXB_vec = np.cross(E.T, b_B.T).T
        v_EXB.append(np.linalg.norm(v_EXB_vec, axis=0))

        
        # GradB drift
        gradB_traj = np.stack([
            interp_gradB_1(traj[0:3].T),
            interp_gradB_2(traj[0:3].T),
            interp_gradB_3(traj[0:3].T)
        ], axis=0)

        v_GradB.append(np.linalg.norm(np.cross(b_B.T, gradB_traj.T), axis=1))


        # Interpolate jacobian of b
        J_b_traj = np.empty((3, 3, np.shape(t)[0]), dtype=float)
        for i in range(3):
            for j in range(3):
                J_b_traj[i,j] = J_b_Interp[i,j](traj[0:3].T)

        # p parallel to b
        p_par = np.einsum('it, it->t', p[:3], b)
        #I think p is normalized with the mass of the particle so it is the same as u (no need to consider mass)


        #Curvature drift: (b.Grad)b
        curvature = np.einsum('ti,tij->jt', b.T, J_b_traj.transpose(2, 0, 1)) # TODO check this carefully

        vc_temp = (p_par**2)[:,None] * np.cross(b_B.T, curvature.T)
        v_c.append(np.linalg.norm(vc_temp, axis=1))


        # (v_E.Grad)b drift
        v_EGrad_B = np.einsum('ti,tij->jt', v_EXB_vec.T, J_b_traj.transpose(2, 0, 1))
        v_v_EGrad_B_temp = (p_par**2)[:,None] * np.cross(b_B.T, v_EGrad_B.T)
        v_v_EGrad_B.append(np.linalg.norm(v_v_EGrad_B_temp, axis=1))


    if unload:
        sim["test_electrons"]["tracks"].unload()
        
    #average for every particle
    avrg_v_EXB = np.mean(np.array(v_EXB), axis=0)
    avrg_v_gradB = np.mean(np.array(v_GradB), axis=0)
    avrg_v_c = np.mean(np.array(v_c), axis=0)
    avrg_v_v_EGrad_B = np.mean(np.array(v_v_EGrad_B), axis=0)

    if fig is None:
        fig = plt.figure(figsize=(10, 7))

    if ax is None:
        ax = fig.add_subplot(111)

    ax.plot(t, avrg_v_EXB, label=r"$v_{\mathbf{E} \times \mathbf{B}}$", marker='o', linestyle='-', markersize = 1.8, linewidth=1)
    ax.plot(t, avrg_v_gradB, label=r"$ v_{\nabla \mathbf{B}} / \mu $", marker='o', linestyle='-', markersize = 1.8, linewidth=1)
    ax.plot(t, avrg_v_c, label=r"$v_{\mathbf{c}} : v_{\parallel}^2 \frac{\mathbf{b}}{B}\times (\mathbf{b} \cdot \nabla) \mathbf{b}$", marker='o', linestyle='-', markersize = 1.8, linewidth=1)
    ax.plot(t, avrg_v_v_EGrad_B, label=r"$v_{\parallel} \frac{\mathbf{b}}{B} \times (\mathbf{v_{E}}  \cdot \nabla) \mathbf{b}$", marker='o', linestyle='-', markersize = 1.8, linewidth=1)

    # Labels and title
    ax.set_xlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))
    ax.set_ylabel('Drift velocity' + "$[c]$")
    ax.legend()

    return fig, ax

In [ ]:
# Boris pusher trajectories
particle = range(0, 1000)
#particle = [400]
tmax = 700

fig, ax = None, None                            
fig, ax = plot_drifts(sim_Boris_0_1, particles=particle, tmax=tmax, fig=fig, ax=ax)

ax.set_yscale('log')
ax.set_title("Particles " + str(particle))
plt.tight_layout()
plt.show()

In [ ]:
# test grad B
sim_gradB = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/test_drift_calculatinos/gradB/Boris_1/Boris_1.in")
gradB=get_gradB(sim_gradB, t=0)

print("x")
print(gradB[0][0:3, 0:3 ,0:3].T)

print("y")
print(gradB[1][0:3, 0:3, 0:3].T)

print("z")  
print(gradB[2][0:3 , 0:3, 0:3].T)

In [ ]:
sim_curv = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/test_drift_calculatinos/curvature/Boris_1/Boris_1.in")
sim = sim_curv
particles = [9]
tmin=0
tmax=700

gradB=get_gradB(sim)
interp_gradB_1 = diag_interpolator(gradB[0], t=0)
interp_gradB_2 = diag_interpolator(gradB[1], t=0)
interp_gradB_3 = diag_interpolator(gradB[2], t=0)

J_b = get_Jacobian_b(sim)
J_b_Interp = get_Jacobian_b_Interp(J_b, t=0)

v_EXB=[]
v_GradB=[]
v_c=[]
v_v_EGrad_B=[]
for particle in tqdm(particles, desc="Calculating Drifts"):
    # get x, p, E, B, t for the trajecotry of each particle
    fields = call_silently(get_fields, sim, particle, tmin, tmax)
    traj = call_silently(get_trajectory, sim, particle, tmin, tmax)
    p = call_silently(get_p, sim, particle, tmin, tmax)

    E = fields[0:3, :]
    B = fields[3:6, :]
    b = B / np.linalg.norm(B, axis=0)
    b_B = b / np.linalg.norm(B, axis=0)
    t = fields[-1, :]


    # Interpolate jacobian of b
    J_b_traj = np.empty((3, 3, np.shape(t)[0]), dtype=float)
    for i in range(3):
        for j in range(3):
            J_b_traj[i,j] = J_b_Interp[i,j](traj[0:3].T)

    # p parallel to b
    p_par = np.einsum('it, it->t', p[:3], b)
    #I think p is normalized with the mass of the particle so it is the same as u (no need to consider mass)


    #Curvature drift: (b.Grad)b
    curvature = np.einsum('ti,tij->jt', b.T, J_b_traj.transpose(2, 0, 1)) # TODO check this carefully




In [ ]:
plt.plot(np.linalg.norm(curvature, axis=0), linestyle='-', marker = 'o', markersize = 1.8, linewidth=1, label="Curvature")
plt.plot(1./np.linalg.norm(traj[0:3], axis=0), linestyle='-', marker = 'o', markersize = 1.8, linewidth=1, label="1/R")
plt.legend()
plt.yscale('log')

In [ ]:
# test jacobian b

j_b=get_Jacobian_b(sim_curv)

def dbx_dx(x, y, z):
    return x*y / (x**2+y**2)**(3/2)

def dbx_dy(x, y, z):
    return y**2 / (x**2+y**2)**(3/2) - 1. / (x**2+y**2)**(1/2)

def dbx_dz(x, y, z):
    return 0

def dby_dx(x, y, z):
    return -x**2 / (x**2+y**2)**(3/2) + 1. / (x**2+y**2)**(1/2)

def dby_dy(x, y, z):
    return -x*y / (x**2+y**2)**(3/2)

def dby_dz(x, y, z):
    return 0

def dbz_dx(x, y, z):
    return 0

def dbz_dy(x, y, z):
    return 0

def dbz_dz(x, y, z):
    return 0
j_b_theo = [[dbx_dx, dbx_dy, dbx_dz], [dby_dx, dby_dy, dby_dz], [dbz_dx, dbz_dy, dbz_dz]]


for i, b_buffer in enumerate(["b1", "b2", "b3"]):
    for j, axis in enumerate(["x1", "x2", "x3"]):
        print("d" + b_buffer + "/d" + axis)
        print(j_b[i, j][0][10, 10, 10])
        print("theoretical value:", j_b_theo[i][j](20., 20., 20.))



In [ ]:


# field and charge maps
shot = "000000"
folder_path = Path("/home/exxxx5/Tese/Decks/ComparePushers/test_drift_calculatinos/curvature/Boris_1")

# neg_el = ou.OsirisGridFile(
#     folder_path / ("MS/DENSITY/" + species + "/charge/charge-" + species + "-" + shot + ".h5")
#     )

b1 = ou.OsirisGridFile(
    folder_path / ("MS/FLD/part_b1/part_b1-" + shot + ".h5")
    )

b2 = ou.OsirisGridFile(
    folder_path / ("MS/FLD/part_b2/part_b2-" + shot + ".h5")
    )

b3 = ou.OsirisGridFile(
    folder_path / ("MS/FLD/part_b3/part_b3-" + shot + ".h5")
    )


def plot(data):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Plot for the first subplot
    im1 = ax1.imshow(data.data[:,:,40].T, aspect="auto", origin="lower", cmap="Reds", extent=[data.grid[0][0], data.grid[0][1], data.grid[1][0], data.grid[1][1]])
    cbar1 = fig.colorbar(im1, ax=ax1)
    cbar1.set_label(r"${}$".format(data.label) + r"$\  [{}]$".format(data.units))
    ax1.set_title(r"$t={:.2f}$".format(data.time[0]) + r"$\  [{}]$".format(data.time[1]))
    ax1.set_xlabel(r"${}$".format(data.axis[0]["long_name"]) + r"$\  [{}]$".format(data.axis[0]["units"]))
    ax1.set_ylabel(r"${}$".format(data.axis[1]["long_name"]) + r"$\  [{}]$".format(data.axis[1]["units"]))

    # Plot for the second subplot
    im2 = ax2.imshow(data.data[40,:,:].T, aspect="auto", origin="lower", cmap="Reds", extent=[data.grid[1][0], data.grid[1][1], data.grid[2][0], data.grid[2][1]])
    cbar2 = fig.colorbar(im2, ax=ax2)
    cbar2.set_label(r"${}$".format(data.label) + r"$\  [{}]$".format(data.units))
    ax2.set_title(r"$t={:.2f}$".format(data.time[0]) + r"$\  [{}]$".format(data.time[1]))
    ax2.set_xlabel(r"${}$".format(data.axis[1]["long_name"]) + r"$\  [{}]$".format(data.axis[1]["units"]))
    ax2.set_ylabel(r"${}$".format(data.axis[2]["long_name"]) + r"$\  [{}]$".format(data.axis[2]["units"]))

    return fig, ax1, ax2

plot(b1)